# Numpy入门[19]——结构化数组与记录数组
> 参考：
>
> + [https://ailearning.apachecn.org/](https://ailearning.apachecn.org/)
>
>
> 使用Jupyter进行练习

In [2]:
import numpy as np

## 结构化数组

假设我们要保存这样的数据：

|      | name | age  | wgt  |
| ---- | ---- | ---- | ---- |
| 0    | dan  | 1    | 23.1 |
| 1    | ann  | 0    | 25.1 |
| 2    | sam  | 2    | 8.3  |

希望定义一个一维数组，每个元素有三个属性 `name, age, wgt`，此时我们需要使用结构化数组。

定义数组 `a`：

| 0    | 1    | 2    | 3    |
| ---- | ---- | ---- | ---- |
| 1.0  | 2.0  | 3.0  | 4.0  |

In [3]:
a = np.array([1.0,2.0,3.0,4.0], np.float32)
# 使用 view 方法，将 a 对应的内存按照复数来解释
a.view(np.complex64)

array([1.+2.j, 3.+4.j], dtype=complex64)

| 0    | 1    | 2    | 3    |
| ---- | ---- | ---- | ---- |
| 1.0  | 2.0  | 3.0  | 4.0  |
| real | imag | real | imag |

事实上，可以把复数看成一个结构体，第一部分是实部，第二部分是虚部，这样这个数组便可以看成是一个结构化数组。

换句话说，只需要换种方式解释这段内存，便可以得到结构化数组的效果！

| 0    | 1    | 2    | 3    |
| ---- | ---- | ---- | ---- |
| 1.0  | 2.0  | 3.0  | 4.0  |
| mass | vol  | mass | vol  |

例如，可以将第一个浮点数解释为质量，第二个浮点数解释为速度，则这段内存还可以看成是包含两个域（质量和速度）的结构体。

In [4]:
my_dtype = np.dtype([('mass', 'float32'), ('vol', 'float32')])
a.view(my_dtype)

array([(1., 2.), (3., 4.)], dtype=[('mass', '<f4'), ('vol', '<f4')])

这里，使用 dtype 创造了自定义的结构类型，然后用自定义的结构来解释数组 a 所占的内存。

这里 f4 表示四字节浮点数，< 表示小字节序。

利用这个自定义的结构类型，可以这样初始化结构化数组：

In [5]:
my_data = np.array([(1,1), (1,2), (2,1), (1,3)], my_dtype)

print(my_data)

[(1., 1.) (1., 2.) (2., 1.) (1., 3.)]


In [6]:
print('第一个元素：',my_data[0])
print('使用域的名称来索引速度：',my_data[0]['vol'])
print('得到所有的质量信息：',my_data['mass'])
print('自定义排序规则，先按速度，再按质量：')
my_data.sort(order=('vol', 'mass'))
print(my_data)

第一个元素： (1., 1.)
使用域的名称来索引速度： 1.0
得到所有的质量信息： [1. 1. 2. 1.]
自定义排序规则，先按速度，再按质量：
[(1., 1.) (2., 1.) (1., 2.) (1., 3.)]


回到最初的例子，定义一个人的结构类型：

In [7]:
person_dtype = np.dtype([('name', 'S10'), ('age', 'int'), ('weight', 'float')])
# 查看类型所占字节数：
person_dtype.itemsize

22

In [8]:
# 产生一个 3 x 4 共12人的空结构体数组
people = np.empty((3,4), person_dtype)
# 赋值
people['name'] = [['Brad', 'Jane', 'John', 'Fred'],
                  ['Henry', 'George', 'Brain', 'Amy'],
                  ['Ron', 'Susan', 'Jennife', 'Jill']]
people['age'] = [[33, 25, 47, 54],
                 [29, 61, 32, 27],
                 [19, 33, 18, 54]]
people['weight'] = [[135., 105., 255., 140.],
                    [154., 202., 137., 187.],
                    [188., 135., 88., 145.]]
# 打印
print(people)
print(people[0][0]['name'])

[[(b'Brad', 33, 135.) (b'Jane', 25, 105.) (b'John', 47, 255.)
  (b'Fred', 54, 140.)]
 [(b'Henry', 29, 154.) (b'George', 61, 202.) (b'Brain', 32, 137.)
  (b'Amy', 27, 187.)]
 [(b'Ron', 19, 188.) (b'Susan', 33, 135.) (b'Jennife', 18,  88.)
  (b'Jill', 54, 145.)]]
b'Brad'


### 从文本中读取结构化数组

In [9]:
%%writefile people.txt
name age weight
amy 11 38.2
john 10 40.3
bill 12 21.2

Writing people.txt


利用 `loadtxt` 指定数据类型，从这个文件中读取结构化数组：

In [10]:
person_dtype = np.dtype([('name', 'S10'), ('age', 'int'), ('weight', 'float')])

people = np.loadtxt('people.txt', 
                    skiprows=1,
                    dtype=person_dtype)

people

array([(b'amy', 11, 38.2), (b'john', 10, 40.3), (b'bill', 12, 21.2)],
      dtype=[('name', 'S10'), ('age', '<i4'), ('weight', '<f8')])

In [11]:
import os
# 查看name
people['name']
# 删除文件
os.remove('people.txt')

对于下面的文件： 


In [12]:
%%writefile wood.csv
item,material,number
100,oak,33
110,maple,14
120,oak,7
145,birch,3

Writing wood.csv


定义转换函数处理材料属性，使之对应一个整数：

In [13]:
tree_to_int = dict(oak = 1,
                   maple=2,
                   birch=3)
def convert(s):
    return tree_to_int.get(s.decode('ascii'))

# 使用 genfromtxt 载入数据，可以自动从第一行读入属性名称：
data = np.genfromtxt('wood.csv',
                     delimiter=',', # 逗号分隔
                     dtype=np.int64, # 数据类型
                     names=True,   # 从第一行读入域名
                     converters={1:convert}
                    )
data

array([(100, 1, 33), (110, 2, 14), (120, 1,  7), (145, 3,  3)],
      dtype=[('item', '<i8'), ('material', '<i8'), ('number', '<i8')])

In [14]:
# 查看域
print(data['material'])
# 删除文件
os.remove('wood.csv')

[1 2 1 3]


## 记录数组

记录数组（record array）与结构数组类似：

In [15]:
# 质点类型
partical_dtype = np.dtype([('mass', 'float'), 
                           ('velocity', 'float')])

生成记录数组要使用 `numpy.rec` 里的 `fromrecords` 方法：

In [16]:
from numpy import rec

particals_rec = rec.fromrecords([(1,1), (1,2), (2,1), (1,3)], 
                                dtype = partical_dtype)
particals_rec

rec.array([(1., 1.), (1., 2.), (2., 1.), (1., 3.)],
          dtype=[('mass', '<f8'), ('velocity', '<f8')])

在记录数组中，域可以通过属性来获得：

In [17]:
particals_rec.mass

array([1., 1., 2., 1.])

也可以通过域来查询：

In [18]:
particals_rec['mass']

array([1., 1., 2., 1.])

不过，记录数组的运行效率要比结构化数组要慢一些。

也可以通过将一个结构化数组看成记录数组：

In [19]:
particals = np.array([(1,1), (1,2), (2,1), (1,3)],
                     dtype = partical_dtype)
particals_rec = particals.view(np.recarray)
print(particals_rec.mass)
print(particals_rec.velocity)
# 对于自定义的类型，可以通过它的 names 属性查看它有哪些域
print(particals.dtype.names)

[1. 1. 2. 1.]
[1. 2. 1. 3.]
('mass', 'velocity')
